<a href="https://colab.research.google.com/github/egonik-unlp/tp_curso_ml/blob/main/src/notebooks/nn_transmembrane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/egonik-unlp/tp_curso_ml.git
#!git pull
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from google.colab import drive
from tensorflow.keras.utils import to_categorical
drive.mount('/content/drive')
import os
from datetime import datetime

Cloning into 'tp_curso_ml'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 192 (delta 73), reused 171 (delta 58), pack-reused 0
Receiving objects: 100% (192/192), 114.57 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Mounted at /content/drive


In [3]:
%cd /content/tp_curso_ml/src/pickle/numpy_arrays/
data=np.load('data_compressed.npz', allow_pickle=True)

/content/tp_curso_ml/src/pickle/numpy_arrays


In [4]:
X_1=data.f.seqs
labels_0=data.f.labels_tmd.reshape(-1,1)
labels_1=data.f.labels_ntmd.reshape(-1,1)

In [5]:
def to_categorial_transmembrane(*arrays):
  retval=[]
  for array in arrays:
    retval.append(keras.utils.to_categorical(array))
  return retval

In [6]:
#Labels_0

X_train_val, X_test, t_train_val, t_test_0=train_test_split(X_1, labels_0, random_state=42)
X_train,X_val,t_train_0,t_val_0=train_test_split(X_train_val, t_train_val, random_state=42)

#Labels_1

X_train_val, X_test, t_train_val, t_test_1=train_test_split(X_1, labels_1, random_state=42)
X_train,X_val,t_train_1,t_val_1=train_test_split(X_train_val, t_train_val, random_state=42)

t_train_1_ohe=to_categorical(t_train_1)
t_val_1_ohe=to_categorical(t_val_1)
t_test_1_ohe=to_categorical(t_test_1)

In [7]:
tf.keras.backend.clear_session()
input_=keras.layers.Input(shape=(2000,1))
conv1=keras.layers.Conv1D(filters=32, kernel_size=3,activation='relu')(input_)
conv2=keras.layers.Conv1D(filters=32, kernel_size=3,activation='relu')(conv1)
pool1=keras.layers.MaxPooling1D(pool_size=4)(conv2)
conv3=keras.layers.Conv1D(filters=64, kernel_size=4,activation='relu')(pool1)
conv4=keras.layers.Conv1D(filters=64, kernel_size=4,activation='relu')(conv3)
pool2=keras.layers.MaxPooling1D(pool_size=4)(conv4)
flatten=keras.layers.Flatten()(pool2)
dense1=keras.layers.Dense(128, activation='relu')(flatten)
output_=keras.layers.Dense(1, activation='sigmoid')(dense1)
model=keras.models.Model(inputs=[input_], outputs=[output_])

In [8]:
folder_name=datetime.strftime(datetime.now(), '%d-%m_%H-%M')
os.mkdir('/content/drive/MyDrive/protein_classifier/{}'.format(folder_name))

earlystoppingcb=keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint=keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/protein_classifier/{}/checkpoint.h5'.format(folder_name),save_best_only=True)

In [9]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


In [10]:
history=model.fit(X_train, t_train_0, epochs=1300,batch_size=64, callbacks=[earlystoppingcb, checkpoint], validation_data=(X_val, t_val_0))


Epoch 1/1300
1378/1378 [==============================] - 62s 24ms/step - loss: 0.6785 - accuracy: 0.5786 - val_loss: 0.6678 - val_accuracy: 0.5681
Epoch 2/1300
1378/1378 [==============================] - 32s 23ms/step - loss: 0.6380 - accuracy: 0.6329 - val_loss: 0.7585 - val_accuracy: 0.5354
Epoch 3/1300
1378/1378 [==============================] - 32s 23ms/step - loss: 0.6096 - accuracy: 0.6627 - val_loss: 0.5508 - val_accuracy: 0.6884
Epoch 4/1300
1378/1378 [==============================] - 32s 23ms/step - loss: 0.5653 - accuracy: 0.7041 - val_loss: 0.5469 - val_accuracy: 0.7085
Epoch 5/1300
1378/1378 [==============================] - 32s 23ms/step - loss: 0.5006 - accuracy: 0.7555 - val_loss: 0.5122 - val_accuracy: 0.7515
Epoch 6/1300
1378/1378 [==============================] - 32s 23ms/step - loss: 0.4314 - accuracy: 0.8006 - val_loss: 0.3767 - val_accuracy: 0.8415
Epoch 7/1300
1378/1378 [==============================] - 32s 23ms/step - loss: 0.3726 - accuracy: 0.8356 - val_

# Clasificación en tres clases

Intento clasificar las proteinas en tres ubicaciones: mitocondria, superficie celular y lisosoma

In [32]:
data_3=np.load('data.npy')
X,t=data_3[:,:-1],data_3[:,-1]
X_train_val,X_test, t_train_val, t_test=train_test_split(X,t, random_state=42)
X_train,X_val,t_train,t_val=train_test_split(X_train_val, t_train_val, random_state=42)
t_train_ohe=to_categorical(t_train)
t_val_ohe=to_categorical(t_val)
t_test_ohe=to_categorical(t_test)

In [42]:
X_train_bis=X_train.reshape(-1,2000,1)
X_val_bis=X_val.reshape(-1,2000,1)
X_test_bis=X_test.reshape(-1,2000,1)

In [65]:
tf.keras.backend.clear_session()
input_=keras.layers.Input(shape=X_train_bis.shape[1:])
conv1=keras.layers.Conv1D(filters=32, kernel_size=3,activation='relu')(input_)
conv2=keras.layers.Conv1D(filters=32, kernel_size=3,activation='relu')(conv1)
pool1=keras.layers.MaxPooling1D(pool_size=4)(conv2)

conv3=keras.layers.Conv1D(filters=64, kernel_size=4,activation='relu')(pool1)
conv4=keras.layers.Conv1D(filters=64, kernel_size=4,activation='relu')(conv3)
pool2=keras.layers.MaxPooling1D(pool_size=4)(conv4)

conv5=keras.layers.Conv1D(filters=128, kernel_size=7,activation='relu')(pool2)
conv6=keras.layers.Conv1D(filters=128, kernel_size=7,activation='relu')(conv5)
pool3=keras.layers.MaxPooling1D(pool_size=3)(conv6)




flatten=keras.layers.Flatten()(pool3)
dense1=keras.layers.Dense(64, activation='relu')(flatten)
dense2=keras.layers.Dense(64, activation='relu')(dense1)
output_=keras.layers.Dense(3, activation='softmax')(dense2)
model_3=keras.models.Model(inputs=[input_], outputs=[output_])

In [66]:
folder_name=datetime.strftime(datetime.now(), '%d-%m_%H-%M')
os.mkdir('/content/drive/MyDrive/protein_classifier_with_locations/{}'.format(folder_name))

earlystoppingcb=keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint=keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/protein_classifier_with_locations/{}/checkpoint.h5'.format(folder_name),save_best_only=True)

In [67]:
model_3.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


In [68]:
history=model_3.fit(X_train_bis, t_train_ohe, epochs=1300,batch_size=64, callbacks=[earlystoppingcb, checkpoint], validation_data=(X_val, t_val_ohe))


Epoch 1/1300
25/25 [==============================] - 2s 47ms/step - loss: 1.0700 - accuracy: 0.4135 - val_loss: 1.0363 - val_accuracy: 0.4933
Epoch 2/1300
25/25 [==============================] - 1s 33ms/step - loss: 1.0524 - accuracy: 0.4671 - val_loss: 1.0380 - val_accuracy: 0.4704
Epoch 3/1300
25/25 [==============================] - 1s 33ms/step - loss: 1.0457 - accuracy: 0.4633 - val_loss: 1.0170 - val_accuracy: 0.5105
Epoch 4/1300
25/25 [==============================] - 1s 32ms/step - loss: 1.0416 - accuracy: 0.4665 - val_loss: 1.0068 - val_accuracy: 0.4971
Epoch 5/1300
25/25 [==============================] - 1s 33ms/step - loss: 1.0378 - accuracy: 0.4697 - val_loss: 1.0090 - val_accuracy: 0.5220
Epoch 6/1300
25/25 [==============================] - 1s 33ms/step - loss: 1.0314 - accuracy: 0.4742 - val_loss: 1.0163 - val_accuracy: 0.4665
Epoch 7/1300
25/25 [==============================] - 1s 34ms/step - loss: 1.0286 - accuracy: 0.4786 - val_loss: 1.0102 - val_accuracy: 0.4952

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

# Transfer learning desde un Autoencoder preentrenado con todo el proteoma. Mejora?

# GAN

In [ ]:
%cd /content/tp_curso_ml/src/pickle/json_files

with open('codigo_numerico.json') as file:
  cn=json.load(file)
sec_juan="""MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHAEEEANEEDVLGDET
EEEETKPIELPVKEEEPPEKTVDVAAEKKVVKITSEIPQTERMQKRAERFNVPVSLESKK
AARAARFGISSVPTKGLSSDNKPMVNLDKLKERAQRFGLNVSSISRKSEDDEKLKKRKER
FGIVTSSAGTGTTEDTEAKKRKRAERFGIA"""
# arr_sec_juan=np.array(sec
translator=np.vectorize(lambda aa: cn.get(aa))
cn.get("M")
query_juan=translator([letter for letter in sec_juan])
query_juan